In [2]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np
import random

In [3]:
ctx = utils.try_all_gpus()
lenet = CNN_Model('LeNet')


def get_data(data_idx, batch_size):
    data = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_Non-IID\\train_data" + str(data_idx) + ".npy"
    label = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_Non-IID\\train_label" + str(data_idx) + ".npy"
    data = np.load(data)
    label = np.load(label)
    train_data = mx.io.NDArrayIter(data,label,batch_size=batch_size,shuffle=True)
    return train_data

def validate(net):
    mnist = mx.test_utils.get_mnist()
    val_data = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size=100)
    metric = mx.metric.Accuracy()
    val_data.reset()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    name,acc =metric.get()
    return acc

In [4]:
def train_till_acc(net, stop_acc, learning_rate, batch_size, epoch):
    round = 0
    clist = [x for x in range(100)]
    acc_rec = []
    while True:
        random.shuffle(clist)
        for client in clist:
            train_data = get_data(client, batch_size)
            softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
            trainer = trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})
            metric = mx.metric.Accuracy()
            stop_flag = False
            print("Round %d starting"%(round))
            for i in range(epoch):
                train_data.reset()
                for batch in train_data:
                    data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
                    label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
                    outputs = []
                    with ag.record():
                        for x, y in zip(data, label):
                            z = net(x)
                            loss = softmax_cross_entropy_loss(z, y)
                            loss.backward()
                            outputs.append(z)
                    metric.update(label, outputs)
                    trainer.step(batch.data[0].shape[0])
                name, acc = metric.get()
                metric.reset()
                print('training acc at epoch %d: %s=%f'%(i, name, acc))
                if acc >= 1:
                    break
            acc = validate(net)
            print("accuracy in test set %f"%(acc))
            if acc >= stop_acc:
                stop_flag = True
                break
            round += 1
            acc_rec.append(acc)
        if stop_flag:
            print("round: ",round)
            path =  "E:\\PythonProjects\\Mxnet_FederatedLearning\\exp_rec\\" + str(epoch) + "-" + str(batch_size) + "-niid"
            file_handle = open(path,"w")
            file_handle.write(str(acc_rec))
            break

In [8]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 600, 1)

57 starting
training acc at epoch 0: accuracy=0.986667
accuracy in test set 0.973200
Round 2658 starting
training acc at epoch 0: accuracy=0.966667
accuracy in test set 0.974900
Round 2659 starting
training acc at epoch 0: accuracy=0.981667
accuracy in test set 0.975100
Round 2660 starting
training acc at epoch 0: accuracy=0.958333
accuracy in test set 0.976200
Round 2661 starting
training acc at epoch 0: accuracy=0.983333
accuracy in test set 0.976300
Round 2662 starting
training acc at epoch 0: accuracy=0.991667
accuracy in test set 0.975900
Round 2663 starting
training acc at epoch 0: accuracy=0.941667
accuracy in test set 0.975900
Round 2664 starting
training acc at epoch 0: accuracy=0.966667
accuracy in test set 0.974300
Round 2665 starting
training acc at epoch 0: accuracy=0.960000
accuracy in test set 0.975900
Round 2666 starting
training acc at epoch 0: accuracy=0.976667
accuracy in test set 0.976700
Round 2667 starting
training acc at epoch 0: accuracy=0.968333
accuracy in tes

NameError: name 'acc_rec' is not defined

In [5]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 600, 5)

ning acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965000
Round 1399 starting
training acc at epoch 0: accuracy=0.941667
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.995000
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.972800
Round 1400 starting
training acc at epoch 0: accuracy=0.980000
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.975600
Round 1401 starting
training acc at epoch 0: accuracy=1.000000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.975200
Round 1402 starting
training acc at epoch 0: accuracy=0.961667
training acc at epoch 1: accur

In [6]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 50, 1)

d 996 starting
training acc at epoch 0: accuracy=0.993333
accuracy in test set 0.973000
Round 997 starting
training acc at epoch 0: accuracy=0.995000
accuracy in test set 0.975100
Round 998 starting
training acc at epoch 0: accuracy=0.978333
accuracy in test set 0.931600
Round 999 starting
training acc at epoch 0: accuracy=0.993333
accuracy in test set 0.964600
Round 1000 starting
training acc at epoch 0: accuracy=0.998333
accuracy in test set 0.966000
Round 1001 starting
training acc at epoch 0: accuracy=0.998333
accuracy in test set 0.963200
Round 1002 starting
training acc at epoch 0: accuracy=0.988333
accuracy in test set 0.955000
Round 1003 starting
training acc at epoch 0: accuracy=0.983333
accuracy in test set 0.970000
Round 1004 starting
training acc at epoch 0: accuracy=0.991667
accuracy in test set 0.961000
Round 1005 starting
training acc at epoch 0: accuracy=0.985000
accuracy in test set 0.947900
Round 1006 starting
training acc at epoch 0: accuracy=0.981667
accuracy in tes

In [7]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 600, 20)

000
training acc at epoch 5: accuracy=0.990000
training acc at epoch 6: accuracy=0.991667
training acc at epoch 7: accuracy=0.991667
training acc at epoch 8: accuracy=0.993333
training acc at epoch 9: accuracy=0.996667
training acc at epoch 10: accuracy=0.996667
training acc at epoch 11: accuracy=0.996667
training acc at epoch 12: accuracy=0.996667
training acc at epoch 13: accuracy=0.996667
training acc at epoch 14: accuracy=0.996667
training acc at epoch 15: accuracy=0.996667
training acc at epoch 16: accuracy=0.996667
training acc at epoch 17: accuracy=0.996667
training acc at epoch 18: accuracy=0.996667
training acc at epoch 19: accuracy=0.996667
accuracy in test set 0.964600
Round 727 starting
training acc at epoch 0: accuracy=0.998333
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
training acc at epoch 5: accuracy=0.998333
training acc at epoch 6: accuracy=

In [8]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 10, 1)

cy in test set 0.964600
Round 484 starting
training acc at epoch 0: accuracy=0.993333
accuracy in test set 0.948400
Round 485 starting
training acc at epoch 0: accuracy=0.986667
accuracy in test set 0.963700
Round 486 starting
training acc at epoch 0: accuracy=0.986667
accuracy in test set 0.940300
Round 487 starting
training acc at epoch 0: accuracy=0.986667
accuracy in test set 0.957800
Round 488 starting
training acc at epoch 0: accuracy=0.990000
accuracy in test set 0.950500
Round 489 starting
training acc at epoch 0: accuracy=0.993333
accuracy in test set 0.937400
Round 490 starting
training acc at epoch 0: accuracy=0.986667
accuracy in test set 0.933700
Round 491 starting
training acc at epoch 0: accuracy=0.985000
accuracy in test set 0.939500
Round 492 starting
training acc at epoch 0: accuracy=0.991667
accuracy in test set 0.945000
Round 493 starting
training acc at epoch 0: accuracy=0.995000
accuracy in test set 0.949500
Round 494 starting
training acc at epoch 0: accuracy=0.9

In [9]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 50, 5)

t epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.963400
Round 592 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.956500
Round 593 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.949000
Round 594 starting
training acc at epoch 0: accuracy=0.980000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.954100
Round 595 starting
t

In [10]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 50, 20)

000
training acc at epoch 5: accuracy=1.000000
training acc at epoch 6: accuracy=1.000000
training acc at epoch 7: accuracy=1.000000
training acc at epoch 8: accuracy=1.000000
training acc at epoch 9: accuracy=1.000000
training acc at epoch 10: accuracy=1.000000
training acc at epoch 11: accuracy=1.000000
training acc at epoch 12: accuracy=1.000000
training acc at epoch 13: accuracy=1.000000
training acc at epoch 14: accuracy=1.000000
training acc at epoch 15: accuracy=1.000000
training acc at epoch 16: accuracy=1.000000
training acc at epoch 17: accuracy=1.000000
training acc at epoch 18: accuracy=1.000000
training acc at epoch 19: accuracy=1.000000
accuracy in test set 0.952500
Round 487 starting
training acc at epoch 0: accuracy=0.978333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
training acc at epoch 5: accuracy=1.000000
training acc at epoch 6: accuracy=

In [11]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.98, 0.02, 10, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.957400
Round 448 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.930600
Round 449 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.923400
Round 450 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.897800
Round 451 starting
t